# Mathematical Implementation

In [51]:
import csv
import math
import random

def loadcsv(filename):
    lines = csv.reader(open(filename))
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]
    return dataset

In [52]:
def splitDataSet(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return (trainSet, copy)

In [53]:
def seperateByClass(dataset):
    seperated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in seperated):
            seperated[vector[-1]] = []
        seperated[vector[-1]].append(vector)
    return seperated

In [54]:
def mean(numbers):
    return sum(numbers)/float(len(numbers))

In [55]:
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x - avg, 2) for x in numbers]) / float(len(numbers) - 1)
    return math.sqrt(variance)

In [71]:
def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

In [73]:
def summarizedByClass(dataset):
    seperated = seperateByClass(dataset)
    summaries = {}
    for classvalue, instances in seperated.items():
        summaries[classvalue] = summarize(instances)
    return summaries

In [58]:
def calculateProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean, 2)/2*(math.pow(stdev, 2))))
    return (1/(math.sqrt(2*math.pi)*stdev)) * exponent

In [74]:
def calculateClassProbability(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)    
    return probabilities

In [85]:
def predict(summaries, inputVector):
    probabilities = calculateClassProbability(summaries, inputVector)
    bestLabel = None
    bestProb = None
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
        return bestLabel

In [78]:
def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions

In [87]:
def getAccuracy(testSet, predictons):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictons[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

In [97]:
def main():
    filename = "datasets/diabetes.csv"
    splitRatio = 0.72
    dataset = loadcsv(filename)
    trainingSet, testSet = splitDataSet(dataset, splitRatio)
    print('split {0} rows into train = {1} and test = {2} rows'.format(len(dataset), len(trainingSet), len(testSet)))
    
    # prepare model
    summaries = summarizedByClass(trainingSet)
    
    # test model
    predictions = getPredictions(summaries, testSet)
    accuracy = getAccuracy(testSet, predictions)
    print('Accuracy: {0}%'.format(accuracy))
    
main()

split 768 rows into train = 552 and test = 216 rows
Accuracy: 63.888888888888886%


# Library Implementation

In [2]:
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

dataset = datasets.load_iris()
X, y = dataset.data, dataset.target
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.5)

model = GaussianNB()
model.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [3]:
expected = y_test
predicted = model.predict(X_test)

In [4]:
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        29
           1       0.92      0.92      0.92        25
           2       0.90      0.90      0.90        21

    accuracy                           0.95        75
   macro avg       0.94      0.94      0.94        75
weighted avg       0.95      0.95      0.95        75

[[29  0  0]
 [ 0 23  2]
 [ 0  2 19]]
